<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch06/ch06-05-embedded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

ON_COLAB: True
USE_GIT: True
fatal: destination path 'kagglebook' already exists and is not an empty directory.


In [3]:
# ---------------------------------
# ランダムフォレストの特徴量の重要度
# ---------------------------------
# train_xは学習データ、train_yは目的変数
# 欠損値が扱えないため、欠損値を補完したデータを読み込む
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed_onehot.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed_onehot.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
# ---------------------------------
from sklearn.ensemble import RandomForestClassifier

In [4]:
# ランダムフォレスト
clf = RandomForestClassifier(n_estimators=10, random_state=71)
clf.fit(train_x, train_y)
fi = clf.feature_importances_

In [5]:
# 重要度の上位を出力する
idx = np.argsort(fi)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], fi[idx][:5]
print('random forest importance')
print(top_cols, top_importances)

random forest importance
['medical_info_a1' 'weight' 'age' 'medical_info_a2' 'height'] [0.12604874 0.11164059 0.07741062 0.07132529 0.05367491]


In [6]:
# ---------------------------------
# xgboostの特徴量の重要度
# ---------------------------------
# train_xは学習データ、train_yは目的変数
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
# ---------------------------------
import xgboost as xgb

In [7]:
# xgboost
dtrain = xgb.DMatrix(train_x, label=train_y)
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50
model = xgb.train(params, dtrain, num_round)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:02:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [8]:
# 重要度の上位を出力する
fscore = model.get_score(importance_type='total_gain')
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
print('xgboost importance')
print(fscore[:5])

xgboost importance
[('weight', 2678.062744140625), ('medical_info_a1', 2396.2119140625), ('height', 1944.5745849609375), ('age', 1599.1385498046875), ('medical_info_a2', 1127.4130859375)]
